In [29]:
import numpy as  np
times = np.linspace(400,1000,2500)

In [34]:
a  = 2
c = 2
b = 3


delta_x = np.array([np.random.gamma(a*(times[i]**c- times[i-1]**c), b, 1) for i in range(1,len(times)) ])






In [35]:
from scipy.special import loggamma

def neg_log_likehood(params, delta_x, t):
    a, b, c = params[0], params[1], params[2]
    
    #  on verifie que les parametres sont positifs
    if a <= 0 or b <= 0 or c <= 0:
        return 1e10
    
    # on aplatit delta_x
    dx = delta_x.flatten()
    
    try:
        # differences des t^c
        time_diff = t[1:len(t)]**c - t[0:len(t)-1]**c
        
        # les parametres de forme pour chaque observation
        shape = a * time_diff
        
        # on verifie que shape est positif
        if np.any(shape <= 0):
            return 1e10
        
        # on verifie que dx est positif
        if np.any(dx <= 0):
            return 1e10
        
        # la fonction de log-vraisemblance Gamma: somme des log p(x | shape, scale)
        ll = (shape - 1) * np.log(dx) - dx / b - shape * np.log(b) - loggamma(shape)
        
        # on verifie que ll n'a pas de NaN ou d'infini
        if np.any(np.isnan(ll)) or np.any(np.isinf(ll)):
            return 1e10
        
        return -np.sum(ll)
    except:
        return 1e10



In [36]:
from scipy.optimize import minimize

init_guess = [1, 1, 1]  

###pour eviter la divergence 
bounds = [(0.1, 20), (0.1, 20), (0.1, 5)]

# optimisatiion
result = minimize(
    neg_log_likehood,
    init_guess,
    args=(delta_x, times),
    bounds=bounds,
    method='L-BFGS-B'
)

# resultats
a_hat, b_hat, c_hat = result.x
print("Estimated parameters (L-BFGS-B):")
print(f"a = {a_hat:.4f} (true: {a})")
print(f"b = {b_hat:.4f} (true: {b})")
print(f"c = {c_hat:.4f} (true: {c})")
print("Negative log-likelihood at optimum:", result.fun)
print("Success:", result.success)
print("Message:", result.message)

Estimated parameters (L-BFGS-B):
a = 1.8645 (true: 2)
b = 3.1063 (true: 3)
c = 2.0051 (true: 2)
Negative log-likelihood at optimum: 14428.816389725602
Success: True
Message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
